In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [10]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///resources/hawaii.sqlite")

In [11]:
# reflect an existing database into a new model
db = automap_base()

# reflect the tables
db.prepare(engine, reflect=True)

In [12]:
# View all of the classes that automap found
db.classes.keys()

['measurement', 'station']

In [13]:
# Save references to each table
measurement = db.classes.measurement
station = db.classes.station

In [14]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.

In [26]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
date_last_query = session.query(measurement.date).order_by(measurement.date.desc()).first()
date_last = date_last_query[0]
date_last = dt.datetime.strptime(date_last, '%Y-%m-%d').date()

# Calculate the date one year from the last date in data set.
date_last_yr = date_last - dt.timedelta(days=365)
date_last_yr = date_last_yr.strftime('%Y-%m-%d')

# Perform a query to retrieve the data and precipitation scores
prcp_12m_query = session.query(measurement.date, measurement.prcp)\
                    .filter(measurement.date <= date_last)\
                    .filter(measurement.date >= date_last_yr).all()


# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_12m_df = pd.DataFrame(prcp_12m_query, columns=['date', 'precipitation']).set_index('date')

# Sort the dataframe by date
prcp_12m_df_sorted =  prcp_12m_df.sort_index()

# Use Pandas Plotting with Matplotlib to plot the data
pd_bar_prcp_12m = prcp_12m_df_sorted.plot

# ex Present the bar chart
plt.show()



In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_st_summary = prcp_12m_df.describe()
prcp_st_summary

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
no_station = session.query(Measurement.station).distinct().count()
print(f'There are {no_station} stations available in this dataset')

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
most_active = session.query(Measurement.station, Station.name, func.count(Measurement.station))\
                                .group_by(Measurement.station)\
                                .order_by(func.count(Measurement.station).desc())\
                                .first()

print(f'The most active station is station id {most_active[0]}: {most_active[1]}')

# List the stations and the counts in descending order.
active_station = session.query(Measurement.station, Station.name, func.count(Measurement.station))\
                                .filter(Measurement.station == Station.station)\
                                .group_by(Measurement.station)\
                                .order_by(func.count(Measurement.station).desc())\
                                .all()

# ex Save the query results as a Pandas DataFrame and set the index to the date column
active_station_df = pd.DataFrame(active_station, columns=['station_id', 'name', 'no. of data'])


active_station_df

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
act_stn_summary = session.query(Measurement.station, Station.name, func.min(Measurement.tobs), 
                                func.max(Measurement.tobs), func.avg(Measurement.tobs))\
                                .filter(Measurement.station == Station.station)\
                                .group_by(Measurement.station)\
                                .order_by(func.count(Measurement.station).desc())\
                                .filter(Measurement.station == most_active[0])

# Save the query results as a Pandas DataFrame and set the index to the date column
act_stn_summary_df = pd.DataFrame(act_stn_summary, columns=['station_id', 'name', 'min temp', 'max temp', 'avg temp'])

act_stn_summary_df

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
st_12m = session.query(Measurement.tobs)\
                    .filter(Measurement.date <= last_date_str)\
                    .filter(Measurement.date >= yr_ago_date_str)\
                    .filter(Measurement.station == most_active[0])\
                    .all()

# ex Save the query results as a Pandas DataFrame and set the index to the date column
st_12m_df = pd.DataFrame(st_12m, columns=['tobs'])

# ex Plot the results as a histogram
st_12m_df_plot = st_12m_df.plot.hist(bins=12, figsize=(7,5), color='indianred', alpha=0.8)

# ex Set x/y label, title
plt.title(f'TOBs for {most_active[0]}: {most_active[1]}\n(Date: {yr_ago_date_str} to {last_date_str})', fontweight = 'bold', size=14)
plt.xlabel('Temperature (F)', fontweight = 'bold', size=13)
plt.ylabel('Frequency',fontweight = 'bold', size=13)
st_12m_df_plot.set_facecolor('gainsboro')
plt.tight_layout()

# ex Save bar graph to Images folder
st_12m_df_plot.figure.savefig('Images/hist_stUSC00519281.png')

# ex Present the bar chart
plt.show()

# ex To close pyplot
plt.close()

# Close session

In [ ]:
# Close Session
session.close()